*    We examine below how the shrinkage estimators work for
*       a data set drawn from N-variate normal with length T


In [140]:
import numpy as np
import scipy.linalg as la      # we need the package to compute the Cholesky decomposition
 
np.random.seed(1234)         # so that the random numbers will be the same each time running the program

N = 10
T = 30

mu0 = np.array([[1.3, 1, 1.2,  1,  1.1,  1, 0.9, 1, 0.7, 0.8]])   # True mean
mu0 = mu0.T
V0 = np.identity(10)                            # set the covariance matrix to the identity matrix 

L1 = la.cholesky(V0)             # Cholesky decomposition:  V10 = L1'*L1
L = L1.T                           # V0 = L*L',  L=L1'
                                   # This is for a general V0 as L1 is didenty if V0 is identity
    
M = 10000                      #  # of draws or # of times we use the estimation method
R = np.ones((T,N))                   #     define R to store returns
Z1 = np.ones((M,1))               #     M by 1 vector
Z2 = np.ones((M,1)) 
Z3 = np.ones((M,1)) 

for i in range (M):             #   this loop runs the simulation
    
    for t in range (T):          #  generate the data of legnth T 
        e = np.random.randn(N,1)    
        Y = mu0 + np.matmul(L,e)
        R[t,:] = Y.T

    muR = np.mean(R,axis = 0)
    muR = muR.T                             # N by 1
    V = np.cov(R.T)                        # the covariance estimate, N by N
    VI = np.linalg.inv(V)                     # The inverse of V

    eigvals, eigvecs = np.linalg.eig(V)     #  get the eigenvalues and eigenvectors

    lbar = np.mean(eigvals)
    lambda1 = np.max(eigvals)
     
              # The first shrinkage estimator
            
    b1 = np.mean(muR,axis = 0)*np.ones((N,1))  
    muR.shape = (N,1)                   # make sure N by 1
    a1 = muR - b1
    a = np.dot(a1.T, a1)                             #  The term (muR-b1)'*(muR-b1)
    alpha=(1/T)*(N*lbar - 2*lambda1) / a
    Smu1=(1-alpha)*muR + alpha*b1
    
                  # The 2nd shrinkage estimator
                
    onesN = np.ones((N,1))  
    B = np.matmul(onesN.T,VI)
    b11 = np.matrix(onesN.T)*np.matrix(VI)*np.matrix(muR)
    b12 = np.matrix(onesN.T)*np.matrix(VI)*np.matrix(onesN)
    c11 = b11.item()                             # make an array to a scalar ! 
    c12 = b12.item()   
    b2 = (c11/c12) * onesN          
    b = np.dot((muR-b2).T, (muR-b2))                              #  The term (muR-b2)'*(muR-b2)
    alpha = (1/T)*(N*lbar - 2*lambda1)/ b
    Smu2 = (1-alpha)*muR + alpha*b2
    
                  # store the squared errors of the estimates 
                                       
    Z1[i] = np.dot((muR-mu0).T, (muR-mu0))       #  (muR-mu0)'*(muR-mu0)
    Z2[i] = np.dot((Smu1-mu0).T, (Smu1-mu0))    
    Z3[i] = np.dot((Smu2-mu0).T, (Smu2-mu0))   

Err1 = np.sqrt( np.mean(Z1) )                                  
Err2 = np.sqrt( np.mean(Z2) )
Err3 = np.sqrt( np.mean(Z3) )                           
                                         
print('  The mean squared errors to the true mean  \n')
print(Err1,Err2,Err3)

  The mean squared errors to the true mean  

0.5684770078847065 0.4411894348526486 0.4443511166409121
